In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error,median_absolute_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# !pip install tensorflow --upgrade --force-reinstall
# !pip install --upgrade --force-reinstall keras

In [ ]:
# Convert 'Date' column to datetime
def convert_to_datetime(value):
    try:
        return pd.to_datetime(value)
    except ValueError:
        return None


# function to convert string numbers with commas to numeric values
def convert_to_numeric(value):
    try:
        return float(value.replace(',', ''))
    except (ValueError, AttributeError):
        return None


# Convert volume to numeric
def convert_volume(value):
    try:
        if 'M' in value:
            return float(value.replace('M', '')) * 1000000  #millions
        elif 'K' in value:
             return float(value.replace('K', '')) * 1000 #thousands
        else: return float(value)
    except:
        return None


In [ ]:
XAU_EGP_df = pd.read_csv('/content/XAU_EGP Historical Data.csv',usecols=['Date','Price'])
XAU_EGP_df.head()

,Date,Price
0,02/10/2025,"145,830.31"
1,02/09/2025,"144,455.63"
2,02/07/2025,"143,767.77"
3,02/06/2025,"143,513.00"
4,02/05/2025,"144,101.47"


In [ ]:
XAU_EGP_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3943 entries, 0 to 3942
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    3943 non-null   object
 1   Price   3943 non-null   object
dtypes: object(2)
memory usage: 61.7+ KB


In [ ]:
XAU_EGP_df['Date'] = XAU_EGP_df['Date'].apply(convert_to_datetime)
XAU_EGP_df['Price'] = XAU_EGP_df['Price'].apply(convert_to_numeric)
XAU_EGP_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3943 entries, 0 to 3942
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    3943 non-null   datetime64[ns]
 1   Price   3943 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 61.7 KB


In [ ]:
XAU_EGP_df = XAU_EGP_df.rename(columns={'Price': 'XAU_EGP'})

In [ ]:
XAU_EGP_df

,Date,XAU_EGP
0,2025-02-10,145830.31
1,2025-02-09,144455.63
2,2025-02-07,143767.77
3,2025-02-06,143513.00
4,2025-02-05,144101.47
...,...,...
3938,2010-01-07,6163.72
3939,2010-01-06,6204.87
3940,2010-01-05,6112.29
3941,2010-01-04,6131.70


#usd

In [ ]:
USD_Dollar_vs_EGP_pound_df = pd.read_csv('/content/USD_EGP Historical Data.csv')
USD_Dollar_vs_EGP_pound_df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,02/10/2025,50.2925,50.379,50.4372,50.2500,NaN,0.12%
1,02/09/2025,50.2300,50.300,50.3600,50.2175,NaN,-0.04%
2,02/08/2025,50.2500,50.300,50.3000,50.3000,NaN,0.00%
3,02/07/2025,50.2500,50.300,50.3000,50.3000,NaN,0.00%
4,02/06/2025,50.2500,50.330,50.3600,50.2300,NaN,-0.04%


In [ ]:
USD_Dollar_vs_EGP_pound_df = USD_Dollar_vs_EGP_pound_df.drop(USD_Dollar_vs_EGP_pound_df.columns[-5:], axis=1)
USD_Dollar_vs_EGP_pound_df.head()

,Date,Price
0,02/10/2025,50.2925
1,02/09/2025,50.2300
2,02/08/2025,50.2500
3,02/07/2025,50.2500
4,02/06/2025,50.2500


In [ ]:
# Rename the 'Price' column to 'USD_EGP'
USD_Dollar_vs_EGP_pound_df = USD_Dollar_vs_EGP_pound_df.rename(columns={'Price': 'USD_EGP'})
USD_Dollar_vs_EGP_pound_df.head() # display first few rows to show changes


,Date,USD_EGP
0,02/10/2025,50.2925
1,02/09/2025,50.2300
2,02/08/2025,50.2500
3,02/07/2025,50.2500
4,02/06/2025,50.2500


In [ ]:
USD_Dollar_vs_EGP_pound_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4886 entries, 0 to 4885
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Date     4886 non-null   object 
 1   USD_EGP  4886 non-null   float64
dtypes: float64(1), object(1)
memory usage: 76.5+ KB


In [ ]:
USD_Dollar_vs_EGP_pound_df['Date'] = USD_Dollar_vs_EGP_pound_df['Date'].apply(convert_to_datetime)

In [ ]:
USD_Dollar_vs_EGP_pound_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4886 entries, 0 to 4885
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     4886 non-null   datetime64[ns]
 1   USD_EGP  4886 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 76.5 KB


#deposit rate

In [ ]:
!pip install openpyxl
excel_files = ['/content/Overnight deposit and lending rate.xlsx'] #,'/content/Inflations Historical.xlsx'

for excel_file in excel_files:
    try:
        # Read the Excel file into a pandas DataFrame
        df = pd.read_excel(excel_file)  # Assuming csv files

        # Construct the CSV filename by replacing the extension
        csv_filename = excel_file.replace('.xlsx', '.csv')

        # Save the DataFrame to a CSV file
        df.to_csv(csv_filename, index=False)  # index=False prevents writing row indices
        print(f"Successfully converted '{excel_file}' to '{csv_filename}'")
    except FileNotFoundError:
        print(f"Error: File '{excel_file}' not found.")
    except Exception as e:
        print(f"An error occurred while processing '{excel_file}': {e}")

Successfully converted '/content/Overnight deposit and lending rate.xlsx' to '/content/Overnight deposit and lending rate.csv'


In [ ]:
deposit_lending_rate_df = pd.read_csv('/content/Overnight deposit and lending rate.csv')

# Rename columns using the values in the first row
deposit_lending_rate_df.columns = deposit_lending_rate_df.iloc[0]

# Remove the first row as it's now the header
deposit_lending_rate_df = deposit_lending_rate_df[1:]

deposit_lending_rate_df.head()

,Date,Overnight Deposit Rate,Overnight Lending Rate
1,06 Mar 2024,27.25%,28.25%
2,04 Feb 2024,21.25%,22.25%
3,06 Aug 2023,19.25%,20.25%
4,02 Apr 2023,18.25%,19.25%
5,25 Dec 2022,16.25%,17.25%


In [ ]:
deposit_lending_rate_df['Date'] = deposit_lending_rate_df['Date'].apply(convert_to_datetime)
deposit_lending_rate_df.head()

,Date,Overnight Deposit Rate,Overnight Lending Rate
1,2024-03-06,27.25%,28.25%
2,2024-02-04,21.25%,22.25%
3,2023-08-06,19.25%,20.25%
4,2023-04-02,18.25%,19.25%
5,2022-12-25,16.25%,17.25%


In [ ]:
deposit_lending_rate_col = ['Overnight Deposit Rate', 'Overnight Lending Rate']
for col in deposit_lending_rate_col:
  # Remove the percentage sign and convert the 'Overnight Deposit Rate' column to float
  deposit_lending_rate_df[col] = deposit_lending_rate_df[col].str.rstrip('%').astype('float')

In [ ]:
# Create a date range from 2010-01-01 to today
import datetime
current_time = datetime.datetime.now()
date_rng = pd.date_range(start='2010-01-01', end=str(current_time.date()))

# Create a new DataFrame with the specified date range
new_df = pd.DataFrame({'Date': date_rng})
new_df = new_df[::-1]
new_df.reset_index(drop=True, inplace=True)
# Concatenate the new DataFrame with the existing one
deposit_lending_rate_df = pd.concat([deposit_lending_rate_df, new_df], ignore_index=True)

# Sort the DataFrame by 'Date'
deposit_lending_rate_df = deposit_lending_rate_df.sort_values(by='Date')

# Fill NaN values in the last two columns with the previous valid value
deposit_lending_rate_df.fillna(method='ffill', inplace=True)

deposit_lending_rate_df = deposit_lending_rate_df.sort_values(by='Date', ascending=False)
deposit_lending_rate_df.drop(['Overnight Lending Rate'],axis=1,inplace=True)
# Display the first few rows of the updated dataframe
deposit_lending_rate_df.head()

,Date,Overnight Deposit Rate
25,2025-02-13,27.25
26,2025-02-12,27.25
27,2025-02-11,27.25
28,2025-02-10,27.25
29,2025-02-09,27.25


In [ ]:
deposit_lending_rate_df

,Date,Overnight Deposit Rate
25,2025-02-13,27.25
26,2025-02-12,27.25
27,2025-02-11,27.25
28,2025-02-10,27.25
29,2025-02-09,27.25
...,...,...
5543,2010-01-05,NaN
5544,2010-01-04,NaN
5545,2010-01-03,NaN
5546,2010-01-02,NaN


In [ ]:
deposit_lending_rate_df.dropna(inplace=True)

In [ ]:
deposit_lending_rate_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3887 entries, 25 to 24
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    3887 non-null   datetime64[ns]
 1   Overnight Deposit Rate  3887 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 91.1 KB


#phdc stock market data

In [ ]:
phdc_df = pd.read_csv('/content/Palm Hills Develop Stock Price History.csv')
phdc_df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,02/10/2025,6.44,6.40,6.47,6.34,4.25M,0.62%
1,02/09/2025,6.40,6.50,6.56,6.38,5.24M,-1.23%
2,02/06/2025,6.48,6.44,6.48,6.40,3.25M,0.78%
3,02/05/2025,6.43,6.40,6.46,6.33,6.20M,0.47%
4,02/04/2025,6.40,6.35,6.50,6.35,6.72M,0.95%


In [ ]:
phdc_df.drop(phdc_df.columns[6:], axis=1,inplace=True)
phdc_df

,Date,Price,Open,High,Low,Vol.
0,02/10/2025,6.440,6.400,6.470,6.340,4.25M
1,02/09/2025,6.400,6.500,6.560,6.380,5.24M
2,02/06/2025,6.480,6.440,6.480,6.400,3.25M
3,02/05/2025,6.430,6.400,6.460,6.330,6.20M
4,02/04/2025,6.400,6.350,6.500,6.350,6.72M
...,...,...,...,...,...,...
3641,01/10/2010,4.592,4.553,4.682,4.514,1.75M
3642,01/06/2010,4.542,4.575,4.637,4.480,2.31M
3643,01/05/2010,4.575,4.520,4.687,4.492,1.94M
3644,01/04/2010,4.425,4.268,4.520,4.268,3.19M


In [ ]:
phdc_df['Date'] = phdc_df['Date'].apply(convert_to_datetime)
phdc_df['Vol.'] = phdc_df['Vol.'].apply(convert_volume)

# Feature Engneering

RSI

In [ ]:
!pip install pandas-ta
import pandas_ta as ta

phdc_df = phdc_df[::-1]

phdc_df['RSI'] = ta.rsi(phdc_df['Price'])

# MACD

In [ ]:
phdc_df = phdc_df.rename(columns={'Price': 'close'})
phdc_df.ta.macd(append=True)

,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
0,-0.039491,-0.041283,0.001791
1,-0.035883,-0.047995,0.012112
2,-0.026683,-0.050794,0.024111
3,-0.022698,-0.059507,0.036809
4,-0.012191,-0.063877,0.051686
...,...,...,...
3641,NaN,NaN,NaN
3642,NaN,NaN,NaN
3643,NaN,NaN,NaN
3644,NaN,NaN,NaN


In [ ]:
phdc_df.drop(['MACD_12_26_9','MACDh_12_26_9'],axis=1,inplace=True)
phdc_df

,Date,close,Open,High,Low,Vol.,RSI,MACDs_12_26_9
3645,2010-01-03,4.318,4.218,4.358,4.212,557940.0,NaN,NaN
3644,2010-01-04,4.425,4.268,4.520,4.268,3190000.0,NaN,NaN
3643,2010-01-05,4.575,4.520,4.687,4.492,1940000.0,NaN,NaN
3642,2010-01-06,4.542,4.575,4.637,4.480,2310000.0,NaN,NaN
3641,2010-01-10,4.592,4.553,4.682,4.514,1750000.0,NaN,NaN
...,...,...,...,...,...,...,...,...
4,2025-02-04,6.400,6.350,6.500,6.350,6720000.0,42.762290,0.051686
3,2025-02-05,6.430,6.400,6.460,6.330,6200000.0,44.099963,0.036809
2,2025-02-06,6.480,6.440,6.480,6.400,3250000.0,46.350404,0.024111
1,2025-02-09,6.400,6.500,6.560,6.380,5240000.0,43.343729,0.012112


# Moving Average

In [ ]:
phdc_df['SMA_3'] = ta.sma(phdc_df['close'], length=3)
phdc_df

,Date,close,Open,High,Low,Vol.,RSI,MACDs_12_26_9,SMA_3
3645,2010-01-03,4.318,4.218,4.358,4.212,557940.0,NaN,NaN,NaN
3644,2010-01-04,4.425,4.268,4.520,4.268,3190000.0,NaN,NaN,NaN
3643,2010-01-05,4.575,4.520,4.687,4.492,1940000.0,NaN,NaN,4.439333
3642,2010-01-06,4.542,4.575,4.637,4.480,2310000.0,NaN,NaN,4.514000
3641,2010-01-10,4.592,4.553,4.682,4.514,1750000.0,NaN,NaN,4.569667
...,...,...,...,...,...,...,...,...,...
4,2025-02-04,6.400,6.350,6.500,6.350,6720000.0,42.762290,0.051686,6.416667
3,2025-02-05,6.430,6.400,6.460,6.330,6200000.0,44.099963,0.036809,6.390000
2,2025-02-06,6.480,6.440,6.480,6.400,3250000.0,46.350404,0.024111,6.436667
1,2025-02-09,6.400,6.500,6.560,6.380,5240000.0,43.343729,0.012112,6.436667


# مؤشر متوسط الاتجاه ADX

In [ ]:
phdc_df.ta.adx(append=True)
phdc_df.drop(['DMP_14','DMN_14'],axis=1,inplace=True)
phdc_df

,Date,close,Open,High,Low,Vol.,RSI,MACDs_12_26_9,SMA_3,ADX_14
3645,2010-01-03,4.318,4.218,4.358,4.212,557940.0,NaN,NaN,NaN,NaN
3644,2010-01-04,4.425,4.268,4.520,4.268,3190000.0,NaN,NaN,NaN,NaN
3643,2010-01-05,4.575,4.520,4.687,4.492,1940000.0,NaN,NaN,4.439333,NaN
3642,2010-01-06,4.542,4.575,4.637,4.480,2310000.0,NaN,NaN,4.514000,NaN
3641,2010-01-10,4.592,4.553,4.682,4.514,1750000.0,NaN,NaN,4.569667,NaN
...,...,...,...,...,...,...,...,...,...,...
4,2025-02-04,6.400,6.350,6.500,6.350,6720000.0,42.762290,0.051686,6.416667,14.971560
3,2025-02-05,6.430,6.400,6.460,6.330,6200000.0,44.099963,0.036809,6.390000,15.744598
2,2025-02-06,6.480,6.440,6.480,6.400,3250000.0,46.350404,0.024111,6.436667,16.309625
1,2025-02-09,6.400,6.500,6.560,6.380,5240000.0,43.343729,0.012112,6.436667,16.231457


# مؤشر ستوكاستيك (Stochastic Oscillator)

In [ ]:
phdc_df.ta.stoch(append=True)
phdc_df

,Date,close,Open,High,Low,Vol.,RSI,MACDs_12_26_9,SMA_3,ADX_14,STOCHk_14_3_3,STOCHd_14_3_3
3645,2010-01-03,4.318,4.218,4.358,4.212,557940.0,NaN,NaN,NaN,NaN,NaN,NaN
3644,2010-01-04,4.425,4.268,4.520,4.268,3190000.0,NaN,NaN,NaN,NaN,NaN,NaN
3643,2010-01-05,4.575,4.520,4.687,4.492,1940000.0,NaN,NaN,4.439333,NaN,NaN,NaN
3642,2010-01-06,4.542,4.575,4.637,4.480,2310000.0,NaN,NaN,4.514000,NaN,NaN,NaN
3641,2010-01-10,4.592,4.553,4.682,4.514,1750000.0,NaN,NaN,4.569667,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4,2025-02-04,6.400,6.350,6.500,6.350,6720000.0,42.762290,0.051686,6.416667,14.971560,20.084388,32.076394
3,2025-02-05,6.430,6.400,6.460,6.330,6200000.0,44.099963,0.036809,6.390000,15.744598,12.658228,20.974165
2,2025-02-06,6.480,6.440,6.480,6.400,3250000.0,46.350404,0.024111,6.436667,16.309625,18.565401,17.102672
1,2025-02-09,6.400,6.500,6.560,6.380,5240000.0,43.343729,0.012112,6.436667,16.231457,18.565401,16.596343


# مؤشر البولنجر باند (Bollinger Bands)

In [ ]:
phdc_df.ta.bbands(append=True)
phdc_df.drop(['BBB_5_2.0','BBP_5_2.0'],axis=1,inplace=True)
phdc_df
phdc_df = phdc_df[::-1]
phdc_df

,Date,close,Open,High,Low,Vol.,RSI,MACDs_12_26_9,SMA_3,ADX_14,STOCHk_14_3_3,STOCHd_14_3_3,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0
0,2025-02-10,6.440,6.400,6.470,6.340,4250000.0,45.255893,0.001791,6.440000,16.373549,18.987342,18.706048,6.370670,6.4300,6.489330
1,2025-02-09,6.400,6.500,6.560,6.380,5240000.0,43.343729,0.012112,6.436667,16.231457,18.565401,16.596343,6.318786,6.4100,6.501214
2,2025-02-06,6.480,6.440,6.480,6.400,3250000.0,46.350404,0.024111,6.436667,16.309625,18.565401,17.102672,6.312401,6.4320,6.551599
3,2025-02-05,6.430,6.400,6.460,6.330,6200000.0,44.099963,0.036809,6.390000,15.744598,12.658228,20.974165,6.294950,6.4460,6.597050
4,2025-02-04,6.400,6.350,6.500,6.350,6720000.0,42.762290,0.051686,6.416667,14.971560,20.084388,32.076394,6.274309,6.4880,6.701691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3641,2010-01-10,4.592,4.553,4.682,4.514,1750000.0,NaN,NaN,4.569667,NaN,NaN,NaN,4.282282,4.4904,4.698518
3642,2010-01-06,4.542,4.575,4.637,4.480,2310000.0,NaN,NaN,4.514000,NaN,NaN,NaN,NaN,NaN,NaN
3643,2010-01-05,4.575,4.520,4.687,4.492,1940000.0,NaN,NaN,4.439333,NaN,NaN,NaN,NaN,NaN,NaN
3644,2010-01-04,4.425,4.268,4.520,4.268,3190000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
phdc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3646 entries, 0 to 3645
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           3646 non-null   datetime64[ns]
 1   close          3646 non-null   float64       
 2   Open           3646 non-null   float64       
 3   High           3646 non-null   float64       
 4   Low            3646 non-null   float64       
 5   Vol.           3645 non-null   float64       
 6   RSI            3632 non-null   float64       
 7   MACDs_12_26_9  3613 non-null   float64       
 8   SMA_3          3644 non-null   float64       
 9   ADX_14         3619 non-null   float64       
 10  STOCHk_14_3_3  3631 non-null   float64       
 11  STOCHd_14_3_3  3629 non-null   float64       
 12  BBL_5_2.0      3642 non-null   float64       
 13  BBM_5_2.0      3642 non-null   float64       
 14  BBU_5_2.0      3642 non-null   float64       
dtypes: datetime64[ns](1),

#concat all date in one

In [ ]:
# prompt: concat 4 dataframe in 1 by data column

# Assuming your dataframes are named: XAU_EGP_df, USD_Dollar_vs_EGP_pound_df, deposit_lending_rate_df, phdc_df
# Concatenate the DataFrames based on the 'Date' column
all_data = pd.concat([XAU_EGP_df, USD_Dollar_vs_EGP_pound_df, deposit_lending_rate_df, phdc_df], axis=1)

# Display the first few rows of the concatenated DataFrame
all_data

,Date,XAU_EGP,Date,USD_EGP,Date,Overnight Deposit Rate,Date,close,Open,High,...,Vol.,RSI,MACDs_12_26_9,SMA_3,ADX_14,STOCHk_14_3_3,STOCHd_14_3_3,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0
0,2025-02-10,145830.31,2025-02-10,50.2925,2024-03-06,27.25,2025-02-10,6.44,6.40,6.47,...,4250000.0,45.255893,0.001791,6.440000,16.373549,18.987342,18.706048,6.370670,6.430,6.489330
1,2025-02-09,144455.63,2025-02-09,50.2300,2024-02-04,21.25,2025-02-09,6.40,6.50,6.56,...,5240000.0,43.343729,0.012112,6.436667,16.231457,18.565401,16.596343,6.318786,6.410,6.501214
2,2025-02-07,143767.77,2025-02-08,50.2500,2023-08-06,19.25,2025-02-06,6.48,6.44,6.48,...,3250000.0,46.350404,0.024111,6.436667,16.309625,18.565401,17.102672,6.312401,6.432,6.551599
3,2025-02-06,143513.00,2025-02-07,50.2500,2023-04-02,18.25,2025-02-05,6.43,6.40,6.46,...,6200000.0,44.099963,0.036809,6.390000,15.744598,12.658228,20.974165,6.294950,6.446,6.597050
4,2025-02-05,144101.47,2025-02-06,50.2500,2022-12-25,16.25,2025-02-04,6.40,6.35,6.50,...,6720000.0,42.762290,0.051686,6.416667,14.971560,20.084388,32.076394,6.274309,6.488,6.701691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4881,NaT,NaN,2010-01-05,5.4650,NaT,NaN,NaT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4882,NaT,NaN,2010-01-04,5.4760,NaT,NaN,NaT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4883,NaT,NaN,2010-01-03,5.4795,NaT,NaN,NaT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4884,NaT,NaN,2010-01-02,5.4848,NaT,NaN,NaT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
all_data.shape

(4886, 21)

In [ ]:
all_data.head()

,Date,XAU_EGP,Date,USD_EGP,Date,Overnight Deposit Rate,Date,close,Open,High,...,Vol.,RSI,MACDs_12_26_9,SMA_3,ADX_14,STOCHk_14_3_3,STOCHd_14_3_3,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0
0,2025-02-10,145830.31,2025-02-10,50.2925,2024-03-06,27.25,2025-02-10,6.44,6.40,6.47,...,4250000.0,45.255893,0.001791,6.440000,16.373549,18.987342,18.706048,6.370670,6.430,6.489330
1,2025-02-09,144455.63,2025-02-09,50.2300,2024-02-04,21.25,2025-02-09,6.40,6.50,6.56,...,5240000.0,43.343729,0.012112,6.436667,16.231457,18.565401,16.596343,6.318786,6.410,6.501214
2,2025-02-07,143767.77,2025-02-08,50.2500,2023-08-06,19.25,2025-02-06,6.48,6.44,6.48,...,3250000.0,46.350404,0.024111,6.436667,16.309625,18.565401,17.102672,6.312401,6.432,6.551599
3,2025-02-06,143513.00,2025-02-07,50.2500,2023-04-02,18.25,2025-02-05,6.43,6.40,6.46,...,6200000.0,44.099963,0.036809,6.390000,15.744598,12.658228,20.974165,6.294950,6.446,6.597050
4,2025-02-05,144101.47,2025-02-06,50.2500,2022-12-25,16.25,2025-02-04,6.40,6.35,6.50,...,6720000.0,42.762290,0.051686,6.416667,14.971560,20.084388,32.076394,6.274309,6.488,6.701691


In [ ]:
# Drop duplicate columns, keeping the first occurrence
all_data = all_data.loc[:,~all_data.columns.duplicated(keep='first')]

all_data.head()


,Date,XAU_EGP,USD_EGP,Overnight Deposit Rate,close,Open,High,Low,Vol.,RSI,MACDs_12_26_9,SMA_3,ADX_14,STOCHk_14_3_3,STOCHd_14_3_3,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0
0,2025-02-10,145830.31,50.2925,27.25,6.44,6.40,6.47,6.34,4250000.0,45.255893,0.001791,6.440000,16.373549,18.987342,18.706048,6.370670,6.430,6.489330
1,2025-02-09,144455.63,50.2300,21.25,6.40,6.50,6.56,6.38,5240000.0,43.343729,0.012112,6.436667,16.231457,18.565401,16.596343,6.318786,6.410,6.501214
2,2025-02-07,143767.77,50.2500,19.25,6.48,6.44,6.48,6.40,3250000.0,46.350404,0.024111,6.436667,16.309625,18.565401,17.102672,6.312401,6.432,6.551599
3,2025-02-06,143513.00,50.2500,18.25,6.43,6.40,6.46,6.33,6200000.0,44.099963,0.036809,6.390000,15.744598,12.658228,20.974165,6.294950,6.446,6.597050
4,2025-02-05,144101.47,50.2500,16.25,6.40,6.35,6.50,6.35,6720000.0,42.762290,0.051686,6.416667,14.971560,20.084388,32.076394,6.274309,6.488,6.701691


In [ ]:
all_data.shape

(4886, 18)

In [ ]:
all_data.isna().sum()

,0
Date,943
XAU_EGP,943
USD_EGP,0
Overnight Deposit Rate,999
close,1240
Open,1240
High,1240
Low,1240
Vol.,1241
RSI,1254


In [ ]:
all_data.dropna(inplace=True)

In [ ]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3612 entries, 0 to 3612
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    3612 non-null   datetime64[ns]
 1   XAU_EGP                 3612 non-null   float64       
 2   USD_EGP                 3612 non-null   float64       
 3   Overnight Deposit Rate  3612 non-null   float64       
 4   close                   3612 non-null   float64       
 5   Open                    3612 non-null   float64       
 6   High                    3612 non-null   float64       
 7   Low                     3612 non-null   float64       
 8   Vol.                    3612 non-null   float64       
 9   RSI                     3612 non-null   float64       
 10  MACDs_12_26_9           3612 non-null   float64       
 11  SMA_3                   3612 non-null   float64       
 12  ADX_14                  3612 non-null   float64      

In [ ]:
all_data.shape

(3612, 18)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

all_data2 = all_data.copy()
all_data2['Target'] = all_data2['close'].shift(1)  # توقع اليوم التالي
all_data2.dropna(inplace=True)

x = all_data2.drop(['Target','close','Date'], axis=1)

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
x = pd.DataFrame(x_scaled, columns=x.columns)

y = all_data2['Target']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [ ]:
#@ Hisham Easem ML Helper Baz Edition

from sklearn.linear_model import LinearRegression,SGDRegressor,Lasso,Ridge
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,VotingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,median_absolute_error
from xgboost import XGBRegressor
import lightgbm as lgb

LinearRegressionModel = LinearRegression(fit_intercept=True, copy_X=True,n_jobs=-1)
SGDRegressionModel = SGDRegressor(alpha=0.1,random_state=33,penalty='l2',loss = 'huber')
LassoRegressionModel = Lasso(alpha=1.0,random_state=33)
RidgeRegressionModel = Ridge(alpha=1.0,random_state=33)
RandomForestRegressorModel = RandomForestRegressor(n_estimators=1000,max_depth=8, random_state=33)
GBRModel = GradientBoostingRegressor(n_estimators=500,max_depth=7,learning_rate = 1.5 ,random_state=33)
SVRModel = SVR(C = 1.0 ,epsilon=0.1,kernel = 'rbf')
DecisionTreeRegressorModel = DecisionTreeRegressor( max_depth=3,random_state=33)
KNeighborsRegressorModel = KNeighborsRegressor(n_neighbors = 5, weights='uniform',algorithm = 'auto')
xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.01, max_depth=2, random_state=33)
lgb_model = lgb.LGBMRegressor(verbose=0)
ensemble_model = VotingRegressor(estimators=[('rf', RandomForestRegressorModel), ('gb', GBRModel)], weights=[0.5, 0.5])


Models = [LinearRegressionModel,SGDRegressionModel,LassoRegressionModel,RidgeRegressionModel,RandomForestRegressorModel,
          GBRModel,SVRModel,DecisionTreeRegressorModel,KNeighborsRegressorModel,xgb_model,lgb_model,ensemble_model]



mae_dic = {'model_name' : 'model', 'value' : 100}
mse_dic = {'model_name' : 'model', 'value' : 100}
mdse_dic = {'model_name' : 'model', 'value' : 100}

# all_data2 = all_data.copy()
# all_data2['Target'] = all_data2['close'].shift(1)  # توقع اليوم التالي
# all_data2.dropna(inplace=True)

# x = all_data2.drop(['Target','close','Date'], axis=1)
# y = all_data2['Target']
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

for Model in Models :
    print(f'for Model {str(Model).split("(")[0]}')
    Model.fit(x_train, y_train)
    print(f'Train Score is : {Model.score(x_train, y_train)}')
    print(f'Test Score is : {Model.score(x_test, y_test)}')
    y_pred = Model.predict(x_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mdse = median_absolute_error(y_test, y_pred)
    print(f'MAE value is  : {mae}')
    print(f'MSE value is  : {mse}')
    print(f'MdSE value is  : {mdse}')
    print('=================================================')
    for i,n in zip([mae_dic,mse_dic,mdse_dic],[mae,mse,mdse]) :
        if n < i['value'] :
            i['model_name'] = str(Model).split("(")[0]
            i['value'] = n

print(f'best MAE value is {mae_dic["model_name"]} : {mae_dic["value"]}')
print(f'best MSE value is {mse_dic["model_name"]} : {mse_dic["value"]}')
print(f'best MdSE value is {mdse_dic["model_name"]} : {mdse_dic["value"]}')


for Model LinearRegression
Train Score is : 0.9964231867689229
Test Score is : 0.9960531414053108
MAE value is  : 0.04906932900639595
MSE value is  : 0.005817253124760541
MdSE value is  : 0.031320106021549554
for Model SGDRegressor
Train Score is : 0.9940762685250439
Test Score is : 0.994030566747669
MAE value is  : 0.061066566230196316
MSE value is  : 0.00879831476275786
MdSE value is  : 0.03990085740466087
for Model Lasso
Train Score is : 0.2979882555658526
Test Score is : 0.29885544912509576
MAE value is  : 0.8049349591421499
MSE value is  : 1.0334130883163897
MdSE value is  : 0.6718033960211196
for Model Ridge
Train Score is : 0.9963317551787443
Test Score is : 0.9959342389380893
MAE value is  : 0.05013317421437803
MSE value is  : 0.0059925028157215635
MdSE value is  : 0.03319495269572892
for Model RandomForestRegressor
Train Score is : 0.9984658108666616
Test Score is : 0.9950455323058013
MAE value is  : 0.0549787314149994
MSE value is  : 0.007302362621854275
MdSE value is  : 0.03

In [ ]:
def predict(targer_name):
  all_data1 = all_data.copy()
  all_data1['Target'] = all_data1[targer_name].shift(3)  # توقع اليوم التالي
  all_data1.dropna(inplace=True)

  x = all_data1.drop(['Target',targer_name,'Date'], axis=1)

  scaler = StandardScaler()
  x_scaled = scaler.fit_transform(x)
  x = pd.DataFrame(x_scaled, columns=x.columns)

  y = all_data1['Target']
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

  model = LinearRegression()
  model.fit(x_train, y_train)

  p = all_data.drop([targer_name, 'Date'], axis=1)
  input_features = p.iloc[0].values.reshape(1,-1)

  input_features_scaled = scaler.transform(input_features)
  input_features_scaled = pd.DataFrame(input_features_scaled, columns=p.columns)

  return model.predict(input_features_scaled)

In [ ]:
[f'{i} for tomorrow is {predict(i)}' for i in ['close','Open','High','Low']]

['close for tomorrow is [6.45056172]',
 'Open for tomorrow is [6.50856635]',
 'High for tomorrow is [6.61333441]',
 'Low for tomorrow is [6.39302081]']

In [ ]:
all_data

,Date,XAU_EGP,USD_EGP,Overnight Deposit Rate,close,Open,High,Low,Vol.,RSI,MACDs_12_26_9,SMA_3,ADX_14,STOCHk_14_3_3,STOCHd_14_3_3,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0
0,2025-02-10,145830.31,50.2925,27.25,6.440,6.400,6.470,6.340,4250000.0,45.255893,0.001791,6.440000,16.373549,18.987342,18.706048,6.370670,6.4300,6.489330
1,2025-02-09,144455.63,50.2300,21.25,6.400,6.500,6.560,6.380,5240000.0,43.343729,0.012112,6.436667,16.231457,18.565401,16.596343,6.318786,6.4100,6.501214
2,2025-02-07,143767.77,50.2500,19.25,6.480,6.440,6.480,6.400,3250000.0,46.350404,0.024111,6.436667,16.309625,18.565401,17.102672,6.312401,6.4320,6.551599
3,2025-02-06,143513.00,50.2500,18.25,6.430,6.400,6.460,6.330,6200000.0,44.099963,0.036809,6.390000,15.744598,12.658228,20.974165,6.294950,6.4460,6.597050
4,2025-02-05,144101.47,50.2500,16.25,6.400,6.350,6.500,6.350,6720000.0,42.762290,0.051686,6.416667,14.971560,20.084388,32.076394,6.274309,6.4880,6.701691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3608,2011-04-14,8774.80,7.0194,8.75,5.022,5.022,5.056,4.944,2230000.0,54.508685,0.103037,4.938333,20.102851,35.862516,29.441191,4.841845,4.9574,5.072955
3609,2011-04-13,8681.23,7.0194,8.75,4.933,4.911,5.011,4.832,702370.0,49.634975,0.112912,4.919667,21.240338,26.394948,29.506176,4.829710,4.9652,5.100690
3610,2011-04-12,8660.97,7.0194,8.75,4.860,5.028,5.028,4.754,1450000.0,45.160154,0.124306,4.944000,21.911708,26.066109,38.802869,4.813089,5.0098,5.206511
3611,2011-04-11,8726.06,7.0194,8.75,4.966,4.989,5.017,4.894,277380.0,51.306505,0.135316,5.011000,22.661345,36.057472,54.940723,4.892602,5.0824,5.272198


In [ ]:
all_data2 = all_data.copy()
all_data2['Target'] = all_data2['close'].shift(1)  # توقع اليوم التالي
all_data2.drop('close', axis=1,inplace=True)
all_data2.dropna(inplace=True)
all_data2

,Date,XAU_EGP,USD_EGP,Overnight Deposit Rate,Open,High,Low,Vol.,RSI,MACDs_12_26_9,SMA_3,ADX_14,STOCHk_14_3_3,STOCHd_14_3_3,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,Target
1,2025-02-09,144455.63,50.2300,21.25,6.500,6.560,6.380,5240000.0,43.343729,0.012112,6.436667,16.231457,18.565401,16.596343,6.318786,6.4100,6.501214,6.440
2,2025-02-07,143767.77,50.2500,19.25,6.440,6.480,6.400,3250000.0,46.350404,0.024111,6.436667,16.309625,18.565401,17.102672,6.312401,6.4320,6.551599,6.400
3,2025-02-06,143513.00,50.2500,18.25,6.400,6.460,6.330,6200000.0,44.099963,0.036809,6.390000,15.744598,12.658228,20.974165,6.294950,6.4460,6.597050,6.480
4,2025-02-05,144101.47,50.2500,16.25,6.350,6.500,6.350,6720000.0,42.762290,0.051686,6.416667,14.971560,20.084388,32.076394,6.274309,6.4880,6.701691,6.430
5,2025-02-04,142860.53,50.2700,13.25,6.510,6.520,6.290,12180000.0,40.100291,0.067655,6.466667,14.232247,30.179880,41.378616,6.320385,6.5280,6.735615,6.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3608,2011-04-14,8774.80,7.0194,8.75,5.022,5.056,4.944,2230000.0,54.508685,0.103037,4.938333,20.102851,35.862516,29.441191,4.841845,4.9574,5.072955,4.972
3609,2011-04-13,8681.23,7.0194,8.75,4.911,5.011,4.832,702370.0,49.634975,0.112912,4.919667,21.240338,26.394948,29.506176,4.829710,4.9652,5.100690,5.022
3610,2011-04-12,8660.97,7.0194,8.75,5.028,5.028,4.754,1450000.0,45.160154,0.124306,4.944000,21.911708,26.066109,38.802869,4.813089,5.0098,5.206511,4.933
3611,2011-04-11,8726.06,7.0194,8.75,4.989,5.017,4.894,277380.0,51.306505,0.135316,5.011000,22.661345,36.057472,54.940723,4.892602,5.0824,5.272198,4.860


In [ ]:
correlation = all_data2.corr()['Target'].sort_values(ascending=False)
print(correlation)

Target                    1.000000
High                      0.997565
Low                       0.997405
SMA_3                     0.996671
Open                      0.996161
BBM_5_2.0                 0.994852
BBU_5_2.0                 0.992613
BBL_5_2.0                 0.992390
XAU_EGP                   0.439633
USD_EGP                   0.393066
MACDs_12_26_9             0.343129
RSI                       0.204970
Overnight Deposit Rate    0.195097
STOCHd_14_3_3             0.133241
STOCHk_14_3_3             0.125894
Date                      0.072972
Vol.                      0.071246
ADX_14                   -0.056686
Name: Target, dtype: float64


In [ ]:
from sklearn.model_selection import train_test_split
def predict_with_deeplearning(target_name):
  all_data_copy = all_data.copy()
  all_data_copy['Target'] = all_data_copy[target_name].shift(1)  # توقع اليوم التالي
  all_data_copy.dropna(inplace=True)
  early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

  scaler = MinMaxScaler(feature_range=(0, 1))
  data_scaled = scaler.fit_transform(all_data_copy.drop(['Date',target_name,'Target'], axis=1))
  data_scaled = pd.DataFrame(data_scaled,columns = all_data_copy.columns.drop(['Date',target_name,'Target']))

  x = data_scaled
  y = all_data_copy['Target']
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

  model = Sequential()
  model.add(LSTM(8,activation='relu', return_sequences=True, input_shape = (x_train.shape[1], 1)))
  model.add(LSTM(32,activation='relu', return_sequences=True ))
  model.add(LSTM(32,activation='relu', return_sequences=True ))
  model.add(LSTM(16,activation='relu', return_sequences=True ))
  model.add(LSTM(8,activation='relu', return_sequences=True ))
  model.add(LSTM(4,activation='relu', return_sequences=True ))
  model.add(LSTM(4, return_sequences=False))
  model.add(Dense(1))
  model.compile(optimizer='adam', loss='mean_squared_error')
  history = model.fit(x_train, y_train, batch_size=32, epochs=50, callbacks=[early_stopping],validation_data=(x_test, y_test))

  input_features = data_scaled.iloc[0].values.reshape(1,-1)
  model.save('model_lstm_forcasting_v_.keras')

  return model.predict(input_features), history

In [ ]:
[f'{i} for tomorrow is {predict_with_deeplearning(i)}' for i in ['close']]

Epoch 1/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 21s 69ms/step - loss: 6.6508 - val_loss: 2.5748
Epoch 2/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step - loss: 2.2700 - val_loss: 1.9561
Epoch 3/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 11s 62ms/step - loss: 1.8646 - val_loss: 1.6059
Epoch 4/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step - loss: 1.5580 - val_loss: 1.4896
Epoch 5/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - loss: 1.4348 - val_loss: 1.2626
Epoch 6/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 6s 58ms/step - loss: 1.1711 - val_loss: 1.0991
Epoch 7/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 11s 62ms/step - loss: 1.0287 - val_loss: 0.9216
Epoch 8/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step - loss: 0.7966 - val_loss: 0.8318
Epoch 9/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 6s 56ms/step - loss: 0.8034 - val_loss: 0.7328
Epoch 10/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 10s 55ms/step - loss: 0.6427 - val_loss: 0.6048
Epoch 11/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 6s 62ms/step - loss: 0.6298 - val_loss: 0.5428
Epoch 12/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step - loss:

['close for tomorrow is (array([[5.9405026]], dtype=float32), <keras.src.callbacks.history.History object at 0x78c4cad31390>)']

# The close prize for the next day was 6.1 and the pridicted is 5.9